# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import MedianStoppingPolicy


ModuleNotFoundError: No module named 'azureml'

## Dataset




In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory
dataset = TabularDatasetFactory.from_delimited_files("https://www.kaggle.com/lucidlenn/sloan-digital-sky-survey?select=Skyserver_SQL2_27_2018+6_51_39+PM.csv",                                                          validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True,                                                         partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')

ModuleNotFoundError: No module named 'azureml'

In [ ]:
ws = Workspace.from_config()
experiment_name = 'stargazer'

experiment=Experiment(ws, experiment_name)

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "stargazer-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

## Hyperdrive Configuration
from sklearn.model_selection import RandomizedSearchCV
hyperparameters = {'max_features':[None, 'auto', 'sqrt', 'log2'],
                   'max_depth':[None, 1, 5, 10, 15, 20],
                   'min_samples_leaf': [1, 2, 4],
                   'min_samples_split': [2, 5, 10],
                   'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)],
                   'criterion': ['gini', 'entropy']}
rf_random = RandomizedSearchCV(clf, hyperparameters, n_iter = 100, cv = 10, verbose=2, random_state=123, n_jobs = -1)
rf_random.fit(x_train, y_train)
TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        "--c": choice(1, 2, 3, 10),
        "--max_depth": choice(25, 50, 75, 100)
        "--min_samples_leaf'": choice(12, 23, 1, 2)
        "--min_samples_split": choice(2, 12, 24, 20)
        "--min_samples_split": choice(25, 50, 70, 30)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory = '.',
              entry_script='stargazer.py',
              compute_target=cpu_cluster_name)

hyperdrive_run_config = HyperDriveConfig(   hyperparameter_sampling=ps,
                                        policy=policy,
                                        primary_metric_name=primary_metric_name,
                                        primary_metric_goal=primary_metric_goal,
                                        max_total_runs=100,
                                        estimator = est,
                                        max_concurrent_runs=4)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
#Submit your experiment

In [ ]:

hyperdrive_run = experiment.submit(hyperdrive_config)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice

os.makedirs('./model', exist_ok=True)
model = best_run.register_model(model_name='stargazer_model',
                           model_path='outputs/stargazer_model.pkl',
                           tags=run.get_metrics())
print(model.name, model.id, model.version, sep='\t')

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 8, memory_gb = 32)
service = Model.deploy(ws, "stargazer", model, inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://1ccb34b4-fd4f-48e3-aa6b-d9eace6906c0.westus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'peOgw8oz4ZOzealnFbS6dH2rb5tCad8u'


# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "objid": 1.237650e+18,
            "ra": 183.531326,
            "dec": 0.089693,
            "u": 19.47406,
            "g": 17.04240,
            "r": 15.94699,
            "i": 15.50342,
            "z": 15.22531,
            "run": 752,
            "rerun": 301,
            "camcol": 4,
            "field": 267,
            "specobjid": 3.722360e+18,
            "class": "STAR",
            "redshift": -0.000009,
            "plate": 3306,
            "mjd": 54922,
            "fiberid": 491
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())




TODO: In the cell below, print the logs of the web service and delete the service